In [1]:
!pip install earthaccess xarray h5netcdf -q


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\dante.torres_pragma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install scikit-learn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.8 kB 640.0 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 648.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
    --------------------------------------- 0.2/8.9 MB 4.6 MB/s eta 0:00:02
    --------------------------------------- 0.2/8.9 MB 3.3 MB/s eta 0:00:03
   - -------------------------------------- 0.3/8.9 MB 2.2 MB/s eta 0:00:04
   - -------------------------------------- 0.4/8.9 MB 2.5 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/8.9 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 0.9/8.9 MB 3.5 MB/s eta 0:00:03
   ----- ---------------------------------- 1.2/8.9 MB 3.8 MB/s eta 0:00:03
   ------ --------------------------------- 1.5/8.9 MB 4.3 MB/s eta 0:00:02
   ------- 


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\dante.torres_pragma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install earthaccess xarray h5netcdf dask scikit-learn joblib pandas -q


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\dante.torres_pragma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import earthaccess
import xarray as xr

try:
    earthaccess.login()
    print("✅ Authentication successful.")
except Exception as e:
    print(f"❌ Authentication error: {e}")

C:\Users\dante.torres_pragma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Authentication successful.


User: patriciotorres
pass: *********

In [5]:
import earthaccess
import xarray as xr
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import joblib
from datetime import datetime
import os
import shutil
import glob
import warnings

warnings.filterwarnings("ignore")

# --- 1. Geographic and Temporal Configuration ---
AREQUIPA_BOUNDS = {
    'lat_min': -17.5,
    'lat_max': -14.5,
    'lon_min': -75.5,
    'lon_max': -71.0
}
# Date range for data extraction
HISTORICAL_START = "2024-01-01"
HISTORICAL_END = "2024-01-30"

# --- REGIONAL DATA EXTRACTION ---
processing_months = pd.date_range(start=HISTORICAL_START, end=HISTORICAL_END, freq='MS')
all_regional_data = []

print(f"Starting data collection for the Arequipa region from {HISTORICAL_START} to {HISTORICAL_END}...")

for month_start in processing_months:
    month_end = month_start + pd.offsets.MonthEnd(1)
    month_str = month_start.strftime('%Y-%m')
    temp_path_month = f"./temp_data_{month_str}/"

    print(f"   Processing month: {month_str}")

    try:
        os.makedirs(temp_path_month, exist_ok=True)
        results = earthaccess.search_data(
            doi='10.5067/7MCPBJ41Y0K6',
            temporal=(month_start.strftime('%Y-%m-%d'), month_end.strftime('%Y-%m-%d')),
            bounding_box=(AREQUIPA_BOUNDS['lon_min'], AREQUIPA_BOUNDS['lat_min'], AREQUIPA_BOUNDS['lon_max'], AREQUIPA_BOUNDS['lat_max'])
        )

        if not results:
            print(f"     No data found for {month_str}.")
            continue

        earthaccess.download(results, local_path=temp_path_month)

        local_files = glob.glob(os.path.join(temp_path_month, "*.nc4"))
        if not local_files:
            continue

        with xr.open_mfdataset(local_files, engine="h5netcdf", combine='by_coords') as ds_month:
            regional_data = ds_month.sel(
                lat=slice(AREQUIPA_BOUNDS['lat_min'], AREQUIPA_BOUNDS['lat_max']),
                lon=slice(AREQUIPA_BOUNDS['lon_min'], AREQUIPA_BOUNDS['lon_max'])
            )
            df_regional = regional_data.load().to_dataframe()
            all_regional_data.append(df_regional)
            print(f"     Regional data from {month_str} extracted.")

    except Exception as e:
        print(f"     An error occurred in {month_str}: {e}")
    finally:
        if os.path.exists(temp_path_month):
            shutil.rmtree(temp_path_month)

# --- CONSOLIDATION AND FINAL PREPARATION ---
print("\nConsolidating all regional data...")
if not all_regional_data:
    print("No data was collected. Exiting.")
    exit()

historical_df = pd.concat(all_regional_data)
print("Consolidated dataset. Preparing features...")

historical_df = historical_df.reset_index()

historical_df.rename(columns={'time': 'timestamp'}, inplace=True)
historical_df['hour'] = historical_df['timestamp'].dt.hour
historical_df['dayofyear'] = historical_df['timestamp'].dt.dayofyear
historical_df['month'] = historical_df['timestamp'].dt.month
historical_df['will_rain'] = (historical_df['PRECTOTCORR'] > 0.1).astype(int)
historical_df.dropna(inplace=True)

print("Calculating historical averages for the prediction function...")
historical_means = historical_df.groupby(['dayofyear', 'hour', 'lat', 'lon']).mean(numeric_only=True)
historical_means.to_pickle("historical_means_regional.pkl")

# --- REGIONAL MODEL TRAINING ---

features = [
  "TLML",         # Temperature
  "PRECTOTCORR",  # Rainfall Amount
  "QLML",         # Humidity
  "ULML",         # Wind U-Component
  "VLML"          # Wind V-Component
]
X_base = historical_df[features]

y_temp = historical_df['TLML']
y_will_rain_class = historical_df['will_rain']
y_wind_u = historical_df['ULML']
y_wind_v = historical_df['VLML']
y_humidity = historical_df['QLML']

print(f"\nStarting training with {len(X_base)} records...")

# --- Temperature (Regression) ---
print("Training Temperature model...")

X_for_temp = X_base.drop('TLML', axis=1)
model_temp = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, min_samples_leaf=5)
model_temp.fit(X_for_temp, y_temp)
joblib.dump(model_temp, 'regional_model_temperature.joblib')
print("Regional Temperature model saved.")

# --- Rain Probability (Classification) ---
print("\nTraining Rain Probability model...")
X_for_rain = X_base.drop('PRECTOTCORR', axis=1)
model_rain = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, min_samples_leaf=5)
model_rain.fit(X_for_rain, y_will_rain_class)
joblib.dump(model_rain, 'regional_model_rain.joblib')
print("Regional Rain model saved.")

# --- Wind U (Regression) ---
print("\nTraining model for Wind U-Component...")
X_for_wind_u = X_base.drop('ULML', axis=1)
model_wind_u = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, min_samples_leaf=5)
model_wind_u.fit(X_for_wind_u, y_wind_u)
joblib.dump(model_wind_u, 'regional_model_wind_u.joblib')
print("Regional Wind U model saved.")

# --- Wind V (Regression) ---
print("\nTraining model for Wind V-Component...")
X_for_wind_v = X_base.drop('VLML', axis=1)
model_wind_v = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, min_samples_leaf=5)
model_wind_v.fit(X_for_wind_v, y_wind_v)
joblib.dump(model_wind_v, 'regional_model_wind_v.joblib')
print("Regional Wind V model saved.")

# --- Humidity (Regression) ---
print("\nTraining Humidity model...")
X_for_humidity = X_base.drop('QLML', axis=1)
model_humidity = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, min_samples_leaf=5)
model_humidity.fit(X_for_humidity, y_humidity)
joblib.dump(model_humidity, 'regional_model_humidity.joblib')
print("Regional Humidity model saved.")

print("\n--- Training completed and corrected! ---")

Starting data collection for the Arequipa region from 2024-01-01 to 2024-01-30...
   Processing month: 2024-01


QUEUEING TASKS | : 100%|██████████| 31/31 [00:00<00:00, 373.74it/s]
PROCESSING TASKS | : 100%|██████████| 31/31 [12:01<00:00, 23.29s/it]  
COLLECTING RESULTS | : 100%|██████████| 31/31 [00:00<00:00, 1637.80it/s]


     Regional data from 2024-01 extracted.

Consolidating all regional data...
Consolidated dataset. Preparing features...
Calculating historical averages for the prediction function...

Starting training with 31248 records...
Training Temperature model...
Regional Temperature model saved.

Training Rain Probability model...
Regional Rain model saved.

Training model for Wind U-Component...
Regional Wind U model saved.

Training model for Wind V-Component...
Regional Wind V model saved.

Training Humidity model...
Regional Humidity model saved.

--- Training completed and corrected! ---
